In [ ]:
# convert back to dataframe
def graph_to_dataframe_nodes(graph: nx.DiGraph) -> pd.DataFrame:
    data = []
    for node, attrs in graph.nodes(data=True):
        row = {"node": node}
        row.update(attrs)
        data.append(row)
    return pd.DataFrame(data)


nodes_df = graph_to_dataframe_nodes(G)
nodes_df.head(5)

In [ ]:
def graph_to_dataframe_edges(graph: nx.DiGraph) -> pd.DataFrame:
    data = []
    for u, v, attrs in graph.edges(data=True):
        row = {"source": u, "target": v}
        row.update(attrs)
        data.append(row)
    return pd.DataFrame(data)


edges_df = graph_to_dataframe_edges(G)
edges_df.head(5)

In [ ]:
import copy


def write_gexf(G: nx.DiGraph, output_path: str):
    G = copy.deepcopy(G)
    for n, data in G.nodes(data=True):
        for k, v in data.items():
            if isinstance(v, pd.Timestamp):
                data[k] = v.isoformat()
    for u, v, data in G.edges(data=True):
        for k, v in data.items():
            if isinstance(v, pd.Timestamp):
                data[k] = v.isoformat()
    nx.write_gexf(G, output_path)


write_gexf(G, "higgs_twitter_network.gexf")

In [ ]:
import random


def write_network_html(G, filename, date_format="%Y-%m-%d"):
    """
    Create a deep copy of a NetworkX graph, convert Timestamp attributes to strings,
    and export an interactive PyVis HTML visualization.
    """
    # Deep copy to avoid mutating the original
    nodes_sample = random.sample(G.nodes(), 2000)
    G_copy = copy.deepcopy(G.subgraph(nodes_sample))
    # G_copy = copy.deepcopy(G)

    # Convert all pandas.Timestamp attributes to string
    for n, data in G_copy.nodes(data=True):
        for k, v in list(data.items()):
            if isinstance(v, pd.Timestamp):
                data[k] = v.strftime(date_format)

    for u, v, data in G_copy.edges(data=True):
        for k, val in list(data.items()):
            if isinstance(val, pd.Timestamp):
                data[k] = val.strftime(date_format)

    # Create interactive network
    net = Network(directed=True, height="600px", width="100%")
    net.from_nx(G_copy)

    # Add hover info for nodes
    for node in net.nodes:
        node_data = G_copy.nodes[node["id"]]
        time_info = node_data.get("time", "N/A")
        node["title"] = f"Node: {node['id']}<br>Time: {time_info}"

    # Add hover info for edges
    for edge in net.edges:
        u, v = edge["from"], edge["to"]
        edge_data = G_copy.edges[u, v]
        time_info = edge_data.get("time") or edge_data.get("datetime") or "N/A"
        edge["title"] = f"Edge: {u} → {v}<br>Time: {time_info}"

    net.toggle_physics(False)  # freeze layout for faster load
    # net.force_atlas_2based(gravity=-30)  # or use physics tuned to your needs
    net.show_buttons(filter_=["physics"])  # adds control UI in the HTML

    pos = forceatlas2_layout(
        G_copy,
        max_iter=20,  # default is 100, higher values give more precise layout
        jitter_tolerance=1.0,  # default is 1.0, lower is more precise
        scaling_ratio=2.0,  # default is 2.0, higher values spread out the layout
        gravity=1.0,  # default is 1.0, higher values pull nodes towards the center
    )
    for node in net.nodes:
        x, y = pos[node["id"]]
        node["x"] = x * 1000  # scale for better visibility
        node["y"] = y * 1000  # scale for better visibility

    # Write HTML file
    net.write_html(filename)
    return net


write_network_html(G, "higgs_twitter_network.html")